In [ ]:
import json
import random

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.pyplot import cm

from model_graph import ModelGraph
from simulator import Simulator

# Demo run

In [ ]:
graph_dict = json.load(open("graph_dict.json"))

In [ ]:
sim = Simulator(graph_dict=graph_dict)
sim.run()

In [ ]:
# Code to show graph

aggregated = np.sum(sim.state_var, axis=2)

fig, ax = plt.subplots()
ax.plot(sim.time_series, aggregated[:, sim.c_idx["S"]], "g", label="Susceptibles")
ax.plot(sim.time_series, aggregated[:, sim.c_idx["E"]], "r", label="Exposed")
ax.plot(sim.time_series, aggregated[:, sim.c_idx["I"]], "b", label="Infected")
ax.plot(sim.time_series, aggregated[:, sim.c_idx["R"]], "purple", label="Recovered")
ax.legend()
plt.xlabel("Days")
plt.ylabel("Size of compartments")
plt.title("Time series of the compartments")
plt.savefig("compartments.pdf",dpi=300)
plt.show()

In [ ]:
# Code to show graph

fig, ax = plt.subplots()
ax.plot(sim.time_series, sim.state_var[:, sim.c_idx["S"], 0], "g", label="Age group 0-14")
ax.plot(sim.time_series, sim.state_var[:, sim.c_idx["S"], 1], "r", label="Age group 15-59")
ax.plot(sim.time_series, sim.state_var[:, sim.c_idx["S"], 2], "b", label="Age group 60+")
ax.legend()
plt.xlabel("Days")
plt.ylabel("Number of susceptibles")
plt.title("Susceptible compartments by age groups")
plt.savefig("susceptible_age_groups.pdf",dpi=300)
plt.show()

# Multiple runs from a given point

In [ ]:
sim_original = Simulator(graph_dict=graph_dict)
sim_original.run_after_csocs(10)

In [ ]:
time = np.array(sim_original.time_series).searchsorted(35)

In [ ]:
ts_list = []
i_list = []
for i in range(3):
    sim_1 = Simulator(graph_dict=graph_dict)
    sim_1.init_vals = sim_original.state_var[time]
    sim_1.state_var = [sim_1.init_vals]
    sim_1.last_state = sim_original.state_var[time]
    sim_1.create_transition_matrix()
    sim_1.create_transmission_matrix()
    sim_1.run_after_csocs(10)
    
    aggregated = np.sum(sim_1.state_var, axis=2)
    
    ts_list.append(sim_1.time_series[:])
    i_list.append(aggregated[:, sim_1.c_idx["I"]])

In [ ]:
fig, ax = plt.subplots()
aggregated = np.sum(sim_original.state_var, axis=2)
ax.plot(sim_original.time_series[:time], aggregated[:time, sim_original.c_idx["I"]], "b", linewidth=2)

color = cm.Blues(0.5+np.linspace(0, 0.5, len(ts_list)))
for i, c in zip(range(len(ts_list)), color):
    ax.plot(sim_original.time_series[time]+np.array(ts_list[i]), i_list[i], color=c, linewidth=1, label="Run #"+str(i+1))
plt.xlabel("Days")
plt.ylabel("Number of infected people")
plt.title("Halting condition: 10 days after the first peak")
plt.legend()
plt.savefig("stop_after_peak.pdf",dpi=300)
plt.show()

# Contact reduction

In [ ]:
sim_original = Simulator(graph_dict=graph_dict)
sim_original.run()

In [ ]:
time1 = np.array(sim_original.time_series).searchsorted(25)
time2 = np.array(sim_original.time_series).searchsorted(45)

In [ ]:
ts_list = []
i_list = []
for i in range(1,4):
    sim_1 = Simulator(graph_dict=graph_dict)
    sim_1.init_vals = sim_original.state_var[time2]
    sim_1.state_var = [sim_1.init_vals]
    sim_1.last_state = sim_original.state_var[time2]
    sim_1.model.contact_matrix = sim_1.model.contact_matrix * (1 - 0.1 * i)
    sim_1.create_transition_matrix()
    sim_1.create_transmission_matrix()
    sim_1.run()
    
    aggregated = np.sum(sim_1.state_var, axis=2)
    
    ts_list.append(sim_1.time_series[:])
    i_list.append(aggregated[:, sim_1.c_idx["I"]])

In [ ]:
fig, ax = plt.subplots()
aggregated = np.sum(sim_original.state_var, axis=2)
ax.plot(sim_original.time_series[:], aggregated[:, sim_original.c_idx["I"]], "b",
        linewidth=2, label="No contact reduction")

color = cm.Reds(0.5+np.linspace(0, 0.5, len(ts_list)))
for i, c in zip(range(len(ts_list)), color):
    ax.plot(sim_original.time_series[time2]+np.array(ts_list[i]), i_list[i], color=c,
            linewidth=1, label=str((i+1)*10) + "% contact reduction")
plt.xlabel("Days")
plt.ylabel("Number of infected people")
plt.title("Effect of contact reduction")
plt.legend()
plt.savefig("contact_reduction.pdf",dpi=300)
plt.show()